# Training the Deepsea Network
This project incorporates wandb directly inside the YOLOv5 network (wandb support was added by ultralytics). This notebook allows us to retrain the model and see how train time augmentations can improve the networks ability to detect and track objects without increasing the data sample size. 

## Notes
Training using the ml.p3.2xlarge is about 10x faster ml.g4dn.large which makes the p3 more cost effective. 

## Known Issues
When running the training script in quick succession on the g4dn.xlarge, a memory issue may arise. Shut down the running instance and wait for the machine to become available again and restart the program. This seems to be an issue related to YOLOv5 and Sagemaker's use of a RAM partition as "storage."

## Install Dependencies

In [3]:
!pip install -r deepsea-yolov5/yolov5/requirements.txt
!pip install wandb
!wandb login # append wandb login key here

/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Splitting Images into Train and Validate Categories
In order to make the data useable with YOLOv5 we need to split the single images directory into a training set and a validation set. The labels to benchmark against are have the same name as the images they correspond to, but with a .txt filename.

Extract the images found in the tar.gz file into the /opt/ml/input/data directory.
```
deepsea-yolov5
  ├── opt/ml/data
  │   ├── images
  │   └── labels
 ...
 ```
 The files will be mutated into the following structure upon running

In [8]:
!python ./deepsea-yolov5/src/split.py

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
Using default path /root/mgozzi-augment/deepsea-yolov5/src/../opt/ml/input/data
    Creating: /root/mgozzi-augment/deepsea-yolov5/src/../opt/ml/input/data/images/train
    Creating: /root/mgozzi-augment/deepsea-yolov5/src/../opt/ml/input/data/images/val
COMPLETE


 
 ```
deepsea-yolov5
  ├── opt/ml/data
  │   ├── images
  │   │   ├── train
  │   │   │   ├── image1.png
  │   │   │   └── ...
  │   │   └── val
  │   │       ├── image12345.png
  │   │       └── ...
  │   ├── labels
  │   └──...
 ...
 ```
Run the following script in your terminal of choice in the root of the deepsea-yolov5 directory
<code>python .\src\split.py</code>
<br>This will split the image directory into training and validation directories.

## Training For Evolutions
This will train the network and output the results of the generations to wandb. The best generation will be saved to data/hyps folder under the YOLOv5 directory

In [ ]:
# if training for evolutions, set epochs to 10, and evolve to 50
# if you desire to graph the evolution with the hyperparaters, add the --hyp path and change the hyperparameter.yaml file located in opt/ml/input/data/hyp.scratch-low.yaml
!export WANDB_RUN_GROUP="evolution_const_seed" && python ./deepsea-yolov5/yolov5/train.py \
--img=640 \
--data=./deepsea-yolov5/opt/ml/custom_config.yaml  \
--batch=2 \
--weights=yolov5s.pt \
--cfg=./deepsea-yolov5/yolov5/models/yolov5s.yaml \
--project="902005-vaa"\
--cache \
--epochs=10 \
--evolve=30

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
wandb: Currently logged in as: dcline (mbari). Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=./deepsea-yolov5/yolov5/models/yolov5s.yaml, data=./deepsea-yolov5/opt/ml/custom_config.yaml, hyp=deepsea-yolov5/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=2, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=30, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=902005-vaa, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git

### To Train Using Current Hyperparameter File

In [1]:
# Change the contents of the deeosea-yolov5/opt/ml/input/data/hyp.scratch-low.yaml 
# to utilize the best found hyperparameters from the evolutions performed above. 
!export WANDB_RUN_GROUP="hyperparam_const_seed" && python ./deepsea-yolov5/yolov5/train.py \
--img=640 \
--data=./deepsea-yolov5/opt/ml/custom_config.yaml  \
--batch=2 \
--weights=yolov5s.pt \
--cfg=./deepsea-yolov5/yolov5/models/yolov5s.yaml \
--hyp=./deepsea-yolov5/opt/ml/input/data/hyp.scratch-low.yaml \
--project="902005-vaa"\
--cache \
--epochs=17

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
wandb: Currently logged in as: manuelgozzi (mbari). Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=./deepsea-yolov5/yolov5/models/yolov5s.yaml, data=./deepsea-yolov5/opt/ml/custom_config.yaml, hyp=./deepsea-yolov5/opt/ml/input/data/hyp.scratch-low.yaml, epochs=17, batch_size=2, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=902005-vaa, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits

## Running a sweep (doesn't work well yet)

In [ ]:
!pip install wandb

import os
os.environ['WANDB_PROJECT']="902005-vaa"

!wandb sweep deepsea-yolov5/yolov5/utils/loggers/wandb/sweep.yaml

In [ ]:
# replace mbari/.../... with output from command above
!wandb agent mbari/902005-vaa/w8krnvak